In [2]:
# 자동화하기
# 이어서 쓸 때 save_to_file과 append_to_file 함수에 mode의 파라미터를 'a'로 변경해보기
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
from selenium.webdriver.chrome.options import Options
import csv

# 변수 설정
QUERY = ''
search_QUERY = urlencode({'subquery': QUERY}, encoding='utf-8')
URL = f"https://mlbpark.donga.com/mp/b.php?p=1&m=search&b=kbotown&query=%EB%91%90%EC%82%B0&select=spf&{search_QUERY}&subselect=sct&user="

def get_posts(page_num):
    global QUERY
    posts = []
    
    max_retries = 5
    
    for page in range(134, page_num + 1):
        for attempt in range(max_retries):
            try:
                board_link = f"https://mlbpark.donga.com/mp/b.php?p={30 * page+1}&m=search&b=kbotown&query=%EB%91%90%EC%82%B0&select=spf&{search_QUERY}&subselect=sct&user="
                req = requests.get(board_link, timeout=10)
                req.raise_for_status()
                
                soup = BeautifulSoup(req.text, 'html.parser')
                
                tds = soup.find_all('td', {'class': 't_left'})
                for td in tds:
                    post = td.find('a', {'class': 'txt'})
                    if post is not None:
                        posts.append(post['href'])
                        
                print(f"페이지 {page}의 글 링크 수집 완료")
                break  # 성공적으로 수집이 완료되면 루프 종료
            except requests.exceptions.RequestException as e:
                print(f"Request failed, retrying... ({attempt+1}/{max_retries})")
                time.sleep(2)  # 2초 대기 후 재시도
        else:
            print(f"페이지 {page}의 글 링크 수집 실패")
    
    print(f"총 {len(posts)}개의 글 링크를 찾았습니다.")
    
    # 게시글 링크 csv로 저장
    post_file = open(f"MLBPARK_{QUERY}_{page_num}pages_inner_links.csv", mode='w', encoding='utf-8')
    writer = csv.writer(post_file)
    for post in posts:
        writer.writerow([post])
    post_file.close()
    
    return posts

def extract_info(url):
    max_retries = 5
    
    for attempt in range(max_retries):
        try:
            chrome_options = Options()
            chrome_options.add_argument('--headless')  # Headless 모드 활성화
            driver = webdriver.Chrome(options=chrome_options)
            driver.get(url)
            html = driver.page_source
            driver.quit()
            
            soup = BeautifulSoup(html,'html.parser')
            
            site = soup.find('h1', {'class': 'logo'}).find('a').find('img')['title'].strip()
            title = soup.find('div', {'class': 'titles'}).get_text(strip=True)
            user_id = soup.find('span', {'class': 'nick'}).get_text(strip=True)
            post_time = soup.find('div', {'class': 'text3'}).find('span', {'class': 'val'}).get_text(strip=True) + ':00'
            post_time = datetime.strptime(post_time, '%Y-%m-%d %H:%M:%S')
            post = soup.find('div', {'id': 'contentDetail'}).get_text(strip=True)
            view_cnt = int(soup.find('div', {'class': 'text2'}).find_all('span', {'class': 'val'})[1].get_text(strip=True).replace('\n', '').replace('\r', '').replace(',', ''))
            recomm_cnt = int(soup.find('span', {'id': 'likeCnt'}).get_text(strip=True).replace('\n', '').replace('\r', '').replace(',', ''))
            reply_cnt = int(soup.find('span', {'id': 'replyCnt'}).get_text(strip=True).replace('\n', '').replace('\r', '').replace(',', ''))
            reply_content = []
            if reply_cnt != 0:
                replies = soup.find_all('div', {'class': 'txt_box'})
                for reply in replies:
                    reply_author = reply.find('span', {'class': 'name'}).get_text(strip=True)
                    reply_text = reply.find('span', {'class': 're_txt'}).get_text(strip=True).replace('\n', '').replace('\r', '').replace('\t', '')
                    reply_content.append({'author': reply_author, 'text': reply_text})
            
            data = {'site': site, 'title': title, 'user_id': user_id, 'post_time': post_time, 'post': post, 'view_cnt': view_cnt,
                    'recomm_cnt': recomm_cnt, 'reply_cnt': reply_cnt, 'reply_content': reply_content}
            
            # 댓글이 있다면 딕셔너리에 추가
            if reply_cnt != 0:
                for i, reply_info in enumerate(reply_content):
                    data[f'reply_{i+1}_author'] = reply_info['author']
                    data[f'reply_{i+1}_text'] = reply_info['text']
            
            print(url, '완료')
            break  # 성공적으로 크롤링이 완료되면 루프 종료
        except Exception as e:
            print(f"Crawling failed, retrying... ({attempt+1}/{max_retries})")
            time.sleep(2)  # 2초 대기 후 재시도
    else:
        print("Crawling failed after maximum retries.")
    
    return data

def get_contents(posts):
    for post_link in posts:
        content = extract_info(post_link)
        append_to_file(f"MLBPARK_{QUERY}_test.csv", content)
    return print("모든 작업이 완료되었습니다.")

def save_to_file():
    file = open(f'MLBPARK_{QUERY}__test.csv', mode='a', encoding='utf-8')
    writer = csv.writer(file)
    writer.writerow(['site', 'title', 'user_id', 'post_time', 'post', 'view_cnt', 'recomm_cnt', 'reply_cnt', 'reply_author', 'reply_content'])
    file.close()
    return file

def append_to_file(file_name, data):
    file = open(file_name, mode='a', encoding='utf-8', newline='')  # newline 추가
    writer = csv.writer(file)

    # Write main post data to CSV
    main_post_row = [
        data['site'],
        data['title'],
        data['user_id'],
        data['post_time'],
        data['post'],
        data['view_cnt'],
        data['recomm_cnt'],
        data['reply_cnt']
    ]

    # Check if there are replies
    if 'reply_content' in data and data['reply_cnt'] != 0:
        # Write main post data along with reply data to CSV
        for i, reply_info in enumerate(data['reply_content']):
            writer.writerow(main_post_row + [
                reply_info.get('author', ''),  # Reply author
                reply_info.get('text', '')  # Reply text
            ])
    else:
        # Write only main post data to CSV
        writer.writerow(main_post_row + ['']*2)

    file.close()
    return

# 크롤링할 페이지 수 설정
PAGES = 135
# 게시글 링크 수집
post_links = get_posts(PAGES)
# 결과를 저장할 CSV 파일 생성
save_to_file()
# 게시글 정보 크롤링 및 저장
get_contents(post_links)


페이지 134의 글 링크 수집 완료
페이지 135의 글 링크 수집 완료
총 60개의 글 링크를 찾았습니다.
https://mlbpark.donga.com/mp/b.php?id=202311070087155429&p=4021&b=kbotown&m=view&select=spf&query=%EB%91%90%EC%82%B0&subselect=sct&subquery=&user=&site=donga.com 완료


Exception ignored in: <function Service.__del__ at 0x0000024BE3B14310>
Traceback (most recent call last):
  File "C:\Users\82108\anaconda3\envs\new\lib\site-packages\selenium\webdriver\common\service.py", line 189, in __del__
    self.stop()
  File "C:\Users\82108\anaconda3\envs\new\lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\82108\anaconda3\envs\new\lib\site-packages\selenium\webdriver\common\service.py", line 126, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "C:\Users\82108\anaconda3\envs\new\lib\urllib\request.py", line 214, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\82108\anaconda3\envs\new\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "C:\Users\82108\anaconda3\envs\new\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  F

https://mlbpark.donga.com/mp/b.php?id=202311070087155414&p=4021&b=kbotown&m=view&select=spf&query=%EB%91%90%EC%82%B0&subselect=sct&subquery=&user=&site=donga.com 완료
https://mlbpark.donga.com/mp/b.php?id=202311070087155156&p=4021&b=kbotown&m=view&select=spf&query=%EB%91%90%EC%82%B0&subselect=sct&subquery=&user=&site=donga.com 완료
https://mlbpark.donga.com/mp/b.php?id=202311070087155054&p=4021&b=kbotown&m=view&select=spf&query=%EB%91%90%EC%82%B0&subselect=sct&subquery=&user=&site=donga.com 완료
https://mlbpark.donga.com/mp/b.php?id=202311070087154778&p=4021&b=kbotown&m=view&select=spf&query=%EB%91%90%EC%82%B0&subselect=sct&subquery=&user=&site=donga.com 완료
https://mlbpark.donga.com/mp/b.php?id=202311070087154655&p=4021&b=kbotown&m=view&select=spf&query=%EB%91%90%EC%82%B0&subselect=sct&subquery=&user=&site=donga.com 완료
https://mlbpark.donga.com/mp/b.php?id=202311070087154466&p=4021&b=kbotown&m=view&select=spf&query=%EB%91%90%EC%82%B0&subselect=sct&subquery=&user=&site=donga.com 완료
https://ml

In [2]:
pip install fake_useragent

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
test = pd.read_csv('MLBPARK__20240220-0211.csv')
test

In [1]:
import pandas as pd
a = pd.read_csv('MLBPARK_20240227-0305.csv')
a

,site,title,user_id,post_time,post,view_cnt,recomm_cnt,reply_cnt,reply_author,reply_content
0,엠엘비파크,두산김재환 부상이면 언제 복귀할까요,이거나원참,2024-03-05 13:40:00,제목이 곧 내용입니닽추천공유,430,0,3,28최승용,심한부상아니에요 당장 오늘 독립리그 팀 경기도 선발나온다고 했어요
1,엠엘비파크,두산김재환 부상이면 언제 복귀할까요,이거나원참,2024-03-05 13:40:00,제목이 곧 내용입니닽추천공유,430,0,3,곽코,부상이 아니라 원래 좋지 않은 부위라서 관리해준다는 차원이었습니다
2,엠엘비파크,두산김재환 부상이면 언제 복귀할까요,이거나원참,2024-03-05 13:40:00,제목이 곧 내용입니닽추천공유,430,0,3,sojdwjnz,개막에 맞춰 출전 가능합니다 시범경기에도 나올거고요
3,엠엘비파크,두산펌)다저스의 123번으로 보는 양석환 2번의 합리성,네모난지붕,2024-03-05 13:40:00,‌다저스는 이번에 123번을 베츠-오타니-프리먼 순으로 결정했음물론 시즌 중에 달...,288,0,9,잠실,양의지 5번이 제일 멍충한 짓이죠;; 152억 타자에 팀내 최고 타자룰 왜 5번에 ...
4,엠엘비파크,두산펌)다저스의 123번으로 보는 양석환 2번의 합리성,네모난지붕,2024-03-05 13:40:00,‌다저스는 이번에 123번을 베츠-오타니-프리먼 순으로 결정했음물론 시즌 중에 달...,288,0,9,네모난지붕,잠실// 이승엽이 양의지 체력안배때문에 5번 쓰겠다고 했었는데요 오래오래 써야된다고
...,...,...,...,...,...,...,...,...,...,...
1899,엠엘비파크,"두산‘강승호 선제 적시타→브랜든 3실점→김택연 KK 끝내기 위기 탈출’ 두산, 구춘...",28최승용,2024-02-27 16:18:00,https://n.news.naver.com/sports/kbaseball/arti...,916,0,7,김택연™,허경민 에러라 브랜든 비자책임
1900,엠엘비파크,"두산‘강승호 선제 적시타→브랜든 3실점→김택연 KK 끝내기 위기 탈출’ 두산, 구춘...",28최승용,2024-02-27 16:18:00,https://n.news.naver.com/sports/kbaseball/arti...,916,0,7,최준호,"이야 김택연 실책으로 시작된1사1,3루를 kk로 막네요,,역시 보통내기가 아님"
1901,엠엘비파크,"두산‘강승호 선제 적시타→브랜든 3실점→김택연 KK 끝내기 위기 탈출’ 두산, 구춘...",28최승용,2024-02-27 16:18:00,https://n.news.naver.com/sports/kbaseball/arti...,916,0,7,WilCrowe,두산 연습경기 패가 없네요
1902,엠엘비파크,"두산‘강승호 선제 적시타→브랜든 3실점→김택연 KK 끝내기 위기 탈출’ 두산, 구춘...",28최승용,2024-02-27 16:18:00,https://n.news.naver.com/sports/kbaseball/arti...,916,0,7,최준호,"WilCrowe// 죄송합니다이전 소뱅1군에게 패,,"
